## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease       
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(2, 3):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:49<00:00, 20.15it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_2"])
layer_hs_array.shape

(10000, 267264)

## Layer 2 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

# dim_reduced_vecs = layer_hs_array

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 7894)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 7894D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 0.04 s
Outer iteration 0 / 10
  Iteration 19 (1.05 s, search 0.65 s): objective=4922.71 imbalance=1.044 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (2.14 s, search 1.32 s): objective=4920.71 imbalance=1.118 nsplit=0       
Outer iteration 2 / 10
  Iteration 19 (3.21 s, search 1.98 s): objective=4921.47 imbalance=1.100 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (4.27 s, search 2.64 s): objective=4928.03 imbalance=1.091 nsplit=0       
Objective improved: keep new clusters
Outer iteration 4 / 10
  Iteration 19 (5.32 s, search 3.29 s): objective=4919.51 imbalance=1.099 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (6.38 s, search 3.94 s): objective=4916.74 imbalance=1.264 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (7.42 s, search 4.59 s): objective=4901.01 imbalance=1.539 nsplit=0       
Outer iteration 7 / 10
  Iter

4928.03125

In [10]:
kmeans.centroids #cluster centers

array([[ 0.017112  , -0.00487969,  0.01473732, ...,  0.00087953,
         0.01218723,  0.0135426 ],
       [ 0.0214989 , -0.0013115 ,  0.01243807, ..., -0.00012607,
         0.00843788,  0.01333357],
       [ 0.01174534, -0.01089548,  0.01075566, ...,  0.00153992,
         0.01473929,  0.01566073],
       ...,
       [ 0.01232267, -0.00683162,  0.01197605, ..., -0.00328329,
         0.01713051,  0.01194394],
       [ 0.01191194, -0.00974948,  0.00945816, ..., -0.00131915,
         0.0162207 ,  0.01738647],
       [ 0.01478603, -0.00444456,  0.01175744, ...,  0.00415378,
         0.00707317,  0.01581967]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

0.9999998
1.0000002
1.0000001
0.9999999
1.0
1.0
0.9999998
1.0
0.9999997
1.0000001


In [12]:
kmeans.obj #inertia at each iteration

array([3079.79101562, 4755.37988281, 4824.86816406, 4859.81933594,
       4877.60888672, 4888.93798828, 4896.64746094, 4902.54150391,
       4907.16650391, 4910.22705078, 4912.54931641, 4914.40771484,
       4916.00927734, 4917.26464844, 4918.47949219, 4919.53027344,
       4920.61962891, 4921.62109375, 4922.30419922, 4922.70654297,
       3239.22875977, 4741.60253906, 4794.84082031, 4850.12988281,
       4882.41455078, 4893.60888672, 4900.72460938, 4904.76171875,
       4907.21630859, 4909.20214844, 4910.72949219, 4912.08544922,
       4913.23242188, 4914.20507812, 4915.14990234, 4916.14013672,
       4917.21435547, 4918.39697266, 4919.46728516, 4920.71142578,
       3029.66113281, 4754.37304688, 4835.2265625 , 4872.50488281,
       4886.5703125 , 4894.00830078, 4898.36425781, 4901.42236328,
       4904.24609375, 4907.52636719, 4910.31054688, 4912.36962891,
       4913.90771484, 4915.25      , 4916.45703125, 4917.63964844,
       4918.73388672, 4919.63916016, 4920.55712891, 4921.47167

In [13]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [14]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [15]:
pd.Series(classifications).value_counts()

2    1397
4    1251
3    1215
8    1213
9    1195
6    1088
5     806
7     802
1     594
0     439
dtype: int64

In [16]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

2    1397
4    1251
3    1215
8    1213
9    1195
6    1088
5     806
7     802
1     594
0     439
dtype: int64

In [17]:
prompt_ids = df["prompt_id"]
prompt_ids = prompt_ids.to_numpy()
prompt_ids

array([ 1,  1,  1, ..., 10, 10, 10])

In [18]:
# Get most common centroid for each 1000 points (same label)
max_centroid_per_label = [pd.Series(classifications[i * 1000:(i + 1) * 1000]).value_counts().idxmax() for i in range(10)]
max_centroid_per_label

[4, 8, 3, 4, 7, 9, 9, 3, 4, 1]

In [19]:
# Get most common label for each point classified to a centroid (same centroid)
centroid_labels = [np.where(classifications == i)[0] for i in range(10)]
max_label_per_centroid = [pd.Series(prompt_ids[centroid_labels[i]]).value_counts().idxmax() for i in range(10)]
max_label_per_centroid

[10, 10, 3, 3, 9, 4, 2, 5, 2, 7]

In [20]:
max_centroids = [centroid for centroid in max_centroid_per_label for _ in range(1000)]

In [21]:
np.array(max_centroids)

array([4, 4, 4, ..., 1, 1, 1])

In [22]:
label_to_centroid = {idx + 1 : max_centroid_per_label[idx] for idx in range(len(max_centroid_per_label))}

centroid_to_label = {idx : max_label_per_centroid[idx] for idx in range(len(max_label_per_centroid))}

In [23]:
label_to_centroid

{1: 4, 2: 8, 3: 3, 4: 4, 5: 7, 6: 9, 7: 9, 8: 3, 9: 4, 10: 1}

In [24]:
centroid_to_label

{0: 10, 1: 10, 2: 3, 3: 3, 4: 9, 5: 4, 6: 2, 7: 5, 8: 2, 9: 7}

In [25]:
vectorized_map = np.vectorize(centroid_to_label.get)
classifications_to_label = vectorized_map(classifications)

classifications_to_label

array([ 2,  9,  9, ..., 10, 10, 10])

In [26]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications_to_label[i * 1000:(i + 1) * 1000] == (i + 1)))

Prompt 1 Accuracy:  0.0
Prompt 2 Accuracy:  0.539
Prompt 3 Accuracy:  0.594
Prompt 4 Accuracy:  0.192
Prompt 5 Accuracy:  0.255
Prompt 6 Accuracy:  0.0
Prompt 7 Accuracy:  0.195
Prompt 8 Accuracy:  0.0
Prompt 9 Accuracy:  0.307
Prompt 10 Accuracy:  0.991


In [27]:
print(f"Overall Accuracy: ", np.mean(classifications_to_label == prompt_ids))

Overall Accuracy:  0.3073


In [28]:
for i in range(10):
    print(f"Prompt {i + 1} Accuracy: ", np.mean(classifications[i * 1000:(i + 1) * 1000] == max_centroids[i * 1000:(i + 1)*1000]))

Prompt 1 Accuracy:  0.238
Prompt 2 Accuracy:  0.272
Prompt 3 Accuracy:  0.333
Prompt 4 Accuracy:  0.226
Prompt 5 Accuracy:  0.255
Prompt 6 Accuracy:  0.182
Prompt 7 Accuracy:  0.195
Prompt 8 Accuracy:  0.202
Prompt 9 Accuracy:  0.307
Prompt 10 Accuracy:  0.575


In [29]:
print(f"Overall Accuracy: ", np.mean(classifications == max_centroids))

Overall Accuracy:  0.2785


## Measure Conversion Accuracy

In [30]:
label_to_centroid_vectorized_map = np.vectorize(label_to_centroid.get)
centroid_to_label_vectorized_map = np.vectorize(centroid_to_label.get)
prompt_ids_to_centroids = label_to_centroid_vectorized_map(prompt_ids)
max_centroids_to_labels = centroid_to_label_vectorized_map(max_centroids)

In [31]:
np.mean(max_centroids == prompt_ids_to_centroids) #100% - max_centroids is the prompt ids converted to the max centroid corresponding to each label

1.0

In [32]:
np.mean(max_centroids_to_labels == prompt_ids) #<100%

0.6